In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Load dataset
df = pd.read_csv("/content/Crop and fertilizer dataset.csv")

# Drop duplicate rows
df.drop_duplicates(inplace=True)

# Select only numeric columns for calculating the mean
numeric_cols = df.select_dtypes(include=['number']).columns

# Fill missing values using mean for numeric columns only
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean())

# Encode categorical variables (District_Name, Soil_color, Crop, Fertilizer)
label_encoders = {}
for col in ["District_Name", "Soil_color", "Crop", "Fertilizer"]:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le  # Save encoders for decoding predictions later

# Show cleaned data
print(df.head())

   District_Name  Soil_color  Nitrogen  Phosphorus  Potassium   pH  Rainfall  \
0              0           0        75          50        100  6.5      1000   
1              0           0        80          50        100  6.5      1000   
2              0           0        85          50        100  6.5      1000   
3              0           0        90          50        100  6.5      1000   
4              0           0        95          50        100  6.5      1000   

   Temperature  Crop  Fertilizer                          Link  
0           20    11          17  https://youtu.be/2t5Am0xLTOo  
1           20    11          17  https://youtu.be/2t5Am0xLTOo  
2           20    11          17  https://youtu.be/2t5Am0xLTOo  
3           20    11          17  https://youtu.be/2t5Am0xLTOo  
4           20    11          17  https://youtu.be/2t5Am0xLTOo  


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Features & Target
X_irrigation = df[['Rainfall', 'Temperature', 'pH']]
y_irrigation = df['Rainfall'].shift(-1).fillna(df['Rainfall'].mean())  # Predict next rainfall for irrigation timing

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_irrigation, y_irrigation, test_size=0.2, random_state=42)

# Train model
irrigation_model = RandomForestRegressor(n_estimators=100)
irrigation_model.fit(X_train, y_train)

# Evaluate
y_pred = irrigation_model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f"Irrigation Model MAE: {mae:.2f}")


Irrigation Model MAE: 35.30


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Features & Target
X_fertilizer = df[['Nitrogen', 'Phosphorus', 'Potassium', 'pH', 'Soil_color', 'Crop']]
y_fertilizer = df['Fertilizer']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_fertilizer, y_fertilizer, test_size=0.2, random_state=42)

# Train model
fertilizer_model = RandomForestClassifier(n_estimators=100)
fertilizer_model.fit(X_train, y_train)

# Evaluate
y_pred = fertilizer_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Fertilizer Model Accuracy: {accuracy * 100:.2f}%")


Fertilizer Model Accuracy: 83.83%


In [ ]:
# Features & Target
X_crop = df[['Nitrogen', 'Phosphorus', 'Potassium', 'pH', 'Rainfall', 'Temperature', 'Soil_color']]
y_crop = df['Crop']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_crop, y_crop, test_size=0.2, random_state=42)

# Train model
crop_model = RandomForestClassifier(n_estimators=100)
crop_model.fit(X_train, y_train)

# Evaluate
y_pred = crop_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Crop Model Accuracy: {accuracy * 100:.2f}%")


Crop Model Accuracy: 99.56%


In [ ]:
import time
while True:
    print("Retraining models...")

    # Retrain models with new data
    irrigation_model.fit(X_irrigation, y_irrigation)
    fertilizer_model.fit(X_fertilizer, y_fertilizer)
    crop_model.fit(X_crop, y_crop)

    print("Models retrained successfully! Next training in 7 days.")
    time.sleep(7 * 24 * 3600)  # Wait 7 days before next retrain
